## **Classification Challenge**

Wine experts can identify wines from specific vineyards through smell and taste, but the factors that give different wines their individual characteristics are actually based on their chemical composition.

In this challenge, you must train a classification model to analyze the chemical and visual features of wine samples and classify them based on their cultivar (grape variety).

> **Citation**: The data used in this exercise was originally collected by Forina, M. et al.
> 
> PARVUS - An Extendible Package for Data Exploration, Classification and Correlation. Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.
> 
> It can be downloaded from the UCI dataset repository (Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science).

### **Explore the data**

Let's hit the ground running by importing a CSV file of wine data, which consists of 12 numeric features and a classification label with the following classes:

-   **0** (*variety A*)

-   **1** (*variety B*)

-   **2** (*variety C*)

In [ ]:
# Load the required packages into your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(tidymodels)
  library(here)
  library(janitor)
})

# Read the csv file into a tibble
wine_data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/wine.csv", show_col_types = FALSE)

# Print the first 10 rows of the data
wine_data %>% 
  slice_head(n = 10)


Your challenge is to explore the data and train a classification model that achieves an overall *recall* metric of over 0.95 (95%).

Let's kick off our adventure by reformatting the data to make it easier for a model to use it effectively.

**Question 1.**

Starting with the `wine_data`, encode the `WineVariety` column to a categorical variable (factor) with levels set as: 0, 1, 2. 

Fill in the placeholder `....` with the right code.


In [ ]:
# Load the janitor package for cleaning data
library(janitor)

# Clean data a bit
wine_data <- wine_data %>%
  # Encode WineVariety as category
  mutate(....) %>% 
  # Clean column names
  clean_names()


# View data set
wine_data %>% 
  glimpse()


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 1.R")

The goal of data exploration is to try to understand the *relationships* between its attributes; in particular, any apparent correlation between the *features* and the *label* your model will try to predict. One way of doing this is by using data visualization.

Now let's compare the feature distributions for each label value.

**Question 2.**

We'll begin by restructuring the data such that we can easily plot our data as facets, subplots displaying one subset of the data per each. This is done using [`facet_wrap`](https://ggplot2.tidyverse.org/reference/facet_wrap.html)

-   Use `pivot_longer` (to increase the number of rows and decrease the number of columns) such that all the existing column names except *wine_variety* now fall under a new column name called *features* and their corresponding values under a new column name *values*.

Fill in the placeholder `....` with the right code.

In [ ]:
theme_set(theme_light())
# Pivot data to a long format
wine_data_long <- wine_data %>% 
    pivot_longer(....)


# Make a box plot for each predictor feature
wine_data_long %>% 
  ggplot(mapping = aes(x = wine_variety, y = values, fill = features)) +
  geom_boxplot() + 
  facet_wrap(~ features, scales = "free", ncol = 5) +
  scale_color_viridis_d(option = "plasma", end = .7) +
  theme(legend.position = "none")


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 2.R")

What insights about the wine varieties have you derived from the distribution of the different features?

### Split the data for training and validation

Now, time to train some models. It's best practice to hold out some of your data for *testing* in order to get a better estimate of how your models will perform on new data by comparing the predicted labels with the already known labels in the test set.

**Question 3.**

In this section:

-   Make a split specification of `wine_data` such that *70%* goes to training and the rest goes to testing. Save this to a variable name `wine_split`

-   Extract the training and testing sets from `wine_split` and save them in `wine_train` and `wine_test` variable names respectively.

Fill in the placeholder `....` with the right code.

In [ ]:
# Load the Tidymodels packages
library(tidymodels)


# Split data into 70% for training and 30% for testing
set.seed(2056)
wine_split <- wine_data %>% 
  ....(prop = ....)

# Extract the data in each split
wine_train <- ....(wine_split)
wine_test <- ....


# Print the number of cases in each split
cat("Training cases: ", nrow(wine_train), "\n",
    "Test cases: ", nrow(wine_test), sep = "")


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 3.R")

### Data preprocessing with recipes

Now that we have a set of training features and corresponding training label (`wine_variety`), we can fit a multiclass classification algorithm to the data to create a model.

The function `parsnip::multinom_reg()` defines a model that uses linear predictors to predict multiclass data using the multinomial distribution.

**Question 4.**

In this section:

-   Create a multinomial model specification `mr_spec` which uses `nnet` package as its engine and whose mode is set to `classification`. This model has 1 tuning parameters: `penalty`: amount of regularization. The best way to estimate this value is to *tune* it, but for now, set it to an arbitrary value, say *1*.

-   Create a recipe, `wine_recipe`, with `wine_variety` as the outcome and the rest as predictors, and a step that specifies that all the numeric predictors should be normalized.

-   Bundle the model specification and recipe into a workflow, `mr_wflow.

Fill in the placeholder `....` with the right code.


In [ ]:
# Specify a multinomial regression via nnet
mr_spec <- ....(
  penalty = ....,
  engine = ....,
  .... = "classification"
)


# Create a recipe that specifies that predictors should be on the same scale
wine_recipe <- recipe(...., data = wine_train) %>% 
  ....(all_numeric_predictors())

# Bundle recipe and model specification into a workflow
mr_wflow <- workflow(preprocessor = ...., spec = ....)

# Print out workflow
mr_wflow


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 4.R")

Fantastic!

After a workflow has been *specified*, a model can be *trained* using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function.


In [ ]:
# Fit a workflow object
mr_wflow_fit <- mr_wflow %>% 
  fit(data = wine_train)

# Print wf object
mr_wflow_fit


### Evaluate model performance

Good job! We now have a trained workflow. The workflow's printout shows the coefficients learned during training.

**Question 5.**

This allows us to use the model trained by this workflow to predict labels and their corresponding class probabilities for the test set.

In this section:

-   Obtain a tibble containing the actual wine varieties of the test set, the hard class wine variety predictions and their corresponding probability predictions.

Fill in the placeholder `....` with the right code.

In [ ]:
# Obtain predictions on the test set
results <- ....

# Print the results
results %>% 
  slice_head(n = 10)


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 5.R")

Awesome 👏! Time to evaluate how our model performs. Let's look at the confusion matrix for our model.


In [ ]:
# Confusion matrix
results %>% 
  conf_mat(truth = wine_variety, estimate = .pred_class)


The confusion matrix shows the intersection of predicted and actual label values for each class; in simpler terms, the diagonal intersections from top-left to bottom-right indicate the number of correct predictions.

When dealing with multiple classes, it's generally more intuitive to visualize this as a heat map, like this:


In [ ]:
update_geom_defaults(geom = "tile", new = list(color = "black", alpha = 0.7))
# Visualize confusion matrix
results %>% 
  ....(wine_variety, .pred_class) %>% 
  autoplot(type = "heatmap")


The darker squares in the confusion matrix plot indicate high numbers of cases, and you can hopefully see a diagonal line of darker squares indicating cases where the predicted and actual label are the same.

**Question 6.**

The confusion matrix is helpful since it gives rise to other metrics that can help us better evaluate the performance of a classification model. Let's go through some of them:

*Precision: TP/(TP + FP)*, defined as the proportion of predicted positives that are actually positive.

*Recall: TP/(TP + FN)*, defined as the proportion of positive results out of the number of samples which were actually positive.

*Accuracy: TP + TN/(TP + TN + FP + FN)*, the percentage of labels predicted accurately for a sample.

In this section, using the `yardstick` package, obtain the following metrics:

-   Accuracy
-   Precision
-   Recall

Fill in the placeholder `....` with the right code.

In [ ]:
# Statistical summaries for the confusion matrix
accuracy = accuracy(data = results, truth = ...., estimate = .pred_class)


precision = precision(....)

recall = ....

# Print metrics
accuracy %>% 
  bind_rows(precision) %>% 
  bind_rows(recall)


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 6.R")

What do you think of these metrics?

Let's now evaluate the ROC metrics. In the case of a multiclass classification model, a single ROC curve showing true positive rate vs false positive rate is not possible. However, you can use the rates for each class in a One vs Rest (OVR) comparison to create a ROC chart for each class as below:


In [ ]:
# Make a ROC_CURVE
results %>% 
  roc_curve(wine_variety, c(.pred_0, .pred_1, .pred_2)) %>% 
  ggplot(aes(x = 1 - specificity, y = sensitivity, color = .level)) +
  geom_abline(lty = 2, color = "gray80", size = 0.9) +
  geom_path(show.legend = T, alpha = 0.6, size = 1.2) +
  coord_equal()


To quantify the ROC performance, you can calculate an aggregate area under the curve score that is averaged across all of the OVR curves.

**Question 7.** In this section:
 -   Quantify the ROC performance by calculating the aggregate area under the curve score that is averaged across all of the OVR curves.

Fill in the placeholder `....` with the right code.

In [ ]:
# Calculate ROC_AUC
auc <- results %>% 
  roc_auc(....)

auc


Test your answer:

In [ ]:
. <- ottr::check("tests/Question 7.R")

Overall, did your model do a great job in classifying the wine varieties?

### **Use the model with new data observation**

When you're happy with your model's predictive performance, save it and then use it to predict classes for the following two new wine samples:

-   \[13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285\]

-   \[12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520\]


In [ ]:
library(here)
# Save trained workflow
saveRDS(mr_wflow_fit, "wine_mr_model.rds")


Now, we can load it whenever we need it, and use it to predict labels for new data. This is often called *scoring* or *inferencing*.



In [ ]:
# Create a tibble for the new wine samples
new_wines <- c(13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285) %>% 
  rbind(c(12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520)) %>% 
  as_tibble()
names(new_wines) = names(wine_data)[1:13]

new_wines


In [ ]:
# Load the model into the current R session
loaded_model <- readRDS("wine_mr_model.rds")

# Make predictions
predictions <- loaded_model %>% 
  augment(new_data = new_wines)

predictions


That's it for now. Congratulations on finishing this challenge notebook 🎉! The adventure does not stop there, you could try tuning that hyperparameter, or try out a new model entirely.

Happy Learning,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.
